Get data using this code

In [ ]:
import requests
from bs4 import BeautifulSoup
import nltk
from textblob import TextBlob
import pandas as pd
from datetime import datetime

nltk.download('vader_lexicon')

from nltk.sentiment import SentimentIntensityAnalyzer

reviews = []
ratings = []
sentiment_score = []
name = []
review_date =[]

sia = SentimentIntensityAnalyzer()
from_page = 1
to_page = 150
for i in range(from_page, to_page + 1):
    response = requests.get(f"https://www.trustpilot.com/review/www.google.com?page={i}")
    web_page = response.text
    soup = BeautifulSoup(web_page, "html.parser")
    for review in soup.find_all(class_='paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv styles_reviewCard__hcAvl'):

        review_text = review.find('p', class_ = "typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn")
        date_field = review.find('p', class_ = "typography_body-m__xgxZ_ typography_appearance-default__AAY17 typography_color-black__5LYEn")
        rating = review.find('div', class_ ="styles_reviewHeader__iU9Px")
        if review_text != None:
          text = review_text.get_text()
          date_string = date_field.get_text() #YYYY-MM-DD
          date_object = datetime.strptime(date_string.split(': ')[1], '%B %d, %Y').date()
          review_date.append(date_object)
          reviews.append(text)
          ratings.append(rating.get("data-service-review-rating"))
          sentiment_score.append(sia.polarity_scores(text)['compound'])
          name.append("Google")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Add everything to a dataframe

In [ ]:
data = {"Review": reviews, "Ratings": ratings, "Sentiment Score": sentiment_score, "Name": name, "Date": review_date}
df = pd.DataFrame(data)

Writing a function to change the month to a season

In [ ]:
#season month function
def find_season(month):
  season_month = {
            12:'Winter', 1:'Winter', 2:'Winter',
            3:'Spring', 4:'Spring', 5:'Spring',
            6:'Summer', 7:'Summer', 8:'Summer',
            9:'Autumn', 10:'Autumn', 11:'Autumn'}
  return season_month.get(month)

Add the season to the review

In [ ]:
season_list = []
for each_month in df['Date']:
    season = find_season(each_month.month)
    season_list.append(season)
    
df['Season'] = season_list

In [ ]:
df.to_excel("Google.xlsx")

Start here if data is already available

In [ ]:
'''
import requests
from bs4 import BeautifulSoup
import nltk
from textblob import TextBlob
import pandas as pd
from datetime import datetime
from nltk.sentiment import SentimentIntensityAnalyzer
'''

'\nimport requests\nfrom bs4 import BeautifulSoup\nimport nltk\nfrom textblob import TextBlob\nimport pandas as pd\nfrom datetime import datetime\nfrom nltk.sentiment import SentimentIntensityAnalyzer\n'

In [ ]:
#df = pd.read_excel('Netatmo.xlsx')

Add year to the review

In [ ]:
year_list = []
#df.drop("Year", axis =1)
for each_year in df['Date']:
    year_list.append(each_year.year)
    
df['Year'] = year_list

In [ ]:
import re
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [ ]:
df['New_reviews'] = df['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['New_reviews'].head()

0    vast choice of carpets samples and other floor...
1    you only lose money with google ads. all their...
2    can not change my password (which is on a list...
3    i really look up to the fact that google doesn...
4    just had a very large parcel delivery. it was ...
Name: New_reviews, dtype: object

In [ ]:
df['New_reviews'] = df['New_reviews'].str.replace('[^\w\s]','')
df['New_reviews'].head()

<ipython-input-33-cb142dcdf792>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['New_reviews'] = df['New_reviews'].str.replace('[^\w\s]','')


0    vast choice of carpets samples and other floor...
1    you only lose money with google ads all their ...
2    can not change my password which is on a list ...
3    i really look up to the fact that google doesn...
4    just had a very large parcel delivery it was b...
Name: New_reviews, dtype: object

In [ ]:
# REFERENCE : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
df['New_reviews'] = df['New_reviews'].apply(lambda x: remove_emoji(x))

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop = stopwords.words('english')
df['New_reviews'] = df['New_reviews'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [ ]:
def space(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])
df['New_reviews']= df['New_reviews'].apply(space)
df.head(20)

,Review,Ratings,Sentiment Score,Name,Date,Season,Year,New_reviews
0,Vast choice of carpets samples and other floor...,5,0.9627,Google,2023-04-04,Spring,2023,vast choice carpet sample floor lot remnant va...
1,You only lose money with google ads. All their...,1,-0.6721,Google,2023-04-04,Spring,2023,lose money google ad assistant instead help pu...
2,can not change my password (which is on a list...,1,-0.5934,Google,2023-04-02,Spring,2023,change password list due databreach thank goog...
3,I really look up to the fact that google doesn...,5,0.5994,Google,2023-03-29,Spring,2023,really look fact google do not pay taxis love ...
4,Just had a very large parcel delivery. It was...,5,0.8051,Google,2023-04-04,Spring,2023,large parcel delivery designate time driver he...
5,Neal services. Dug up my hall floor . Laid a c...,1,-0.4517,Google,2023-03-07,Spring,2023,neal service dig hall floor lay concrete floor...
6,My Zodiac had a fault which was fixed for free...,5,0.7430,Google,2023-03-30,Spring,2023,zodiac fault fix free warranty thank prompt ef...
7,/Had need of a Taxi at 4pm from B and Q Great ...,5,0.7332,Google,2023-03-28,Spring,2023,need taxi 4 pm b q great yarmouth gorleston se...
8,"Wow, wow, wow. I've just had a full body massa...",5,0.9887,Google,2023-03-31,Spring,2023,wow wow wow I ve full body massage tracy chamb...
9,Today we had five windows fitted by coastal wi...,5,0.9093,Google,2023-03-29,Spring,2023,today five window fit coastal window newton ab...


In [ ]:
string = ""
from nltk.tokenize import word_tokenize
for review in range(len(df)):
  string = string+ " "+df['New_reviews'].iloc[review]



In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
string_token = word_tokenize(string)

In [ ]:
!pip install NRCLex
from nrclex import NRCLex


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
emotion = []
for word in range(len(string_token)):
  emotion.append(NRCLex(string_token[word]))


In [ ]:
emotion_each_token = []
for obj in range(len(emotion)):
  emotion_each_token.append(emotion[obj].affect_list)

# Remove empty Lists from List
res = [ele for ele in emotion_each_token if ele != []]

#Splitting lists to remove them
res = [item[0].split(",") for item in res]

flat_list = []
for sublist in res:
    for item in sublist:
        flat_list.append(item)

In [ ]:
from collections import Counter
count = Counter(flat_list)
count